In [ ]:
# ### 코랩용
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# os.chdir('/content/drive/Othercomputers/내 노트북/gdrive/dppo')
# !pip uninstall -y gym
# !pip install gymnasium stable-baselines3


텐서보드 log 확인

```
tensorboard --logdir ./ppo_tensorboard_logs/
```

[http://localhost:6006](http://localhost:6006)

In [1]:
from drivingppo.environment import WorldEnv
from drivingppo.model import train_start, train_resume
from world_samples import gen_0, gen_env_naive, gen_env_plain, gen_env_obs

import time
def timeCheck(t0):
    t1 = time.time(); dt = t1-t0
    dt = int(dt)
    h = int(dt/3600)
    m = int(dt/60)%60
    s = int(dt%60)
    print(f'##################### {h}:{m:02d}:{s:02d}')
    return t1

t0 = time.time()

# 입문: 거의 직진만
train_start(
    save_path="0",
    gen_env=lambda: WorldEnv(world_generator=gen_0),
    steps=4_0000,
    gamma=0.9,
    ent_coef=0.05,
)
t0 = timeCheck(t0)

# 초보
train_resume(
    model=    '0',
    save_path='1_naive',
    gen_env=lambda: WorldEnv(world_generator=gen_env_naive),
    tb_log=True,
    steps=32_0000,
    lr=5e-4,
    gamma=0.95,
)
t0 = timeCheck(t0)

# 장애물 없이 일반 주행 + 왕초보 장애물
train_resume(
    model=    '1_naive',
    save_path='2_plain',
    gen_env=lambda: WorldEnv(world_generator=gen_env_plain),
    tb_log=True,
    steps=40_0000,
    save_freq=4_0000,
    lr=3e-4,
    gamma=0.99,
    ent_coef=0.0,
)
t0 = timeCheck(t0)

# 장애물 추가
train_resume(
    model=    '2_plain',
    save_path='3_withobs_1',
    gen_env=lambda: WorldEnv(world_generator=gen_env_obs),
    tb_log=True,
    steps=120_0000,
    save_freq=6_0000,
    lr=1e-4,
    gamma=0.992,
    log_std=-0.7,
    ent_coef=0.004,
)
t0 = timeCheck(t0)

train_resume(
    model=    '3_withobs_1',
    save_path='3_withobs_2',
    gen_env=lambda: WorldEnv(world_generator=gen_env_obs),
    tb_log=True,
    steps=200_0000,
    save_freq=5_0000,
    lr=2e-5,
    gamma=0.99,
    ent_coef=0.0,
)
t0 = timeCheck(t0)